## Practice 1

In [10]:
import re
from collections import Counter
import os

class Cypher:

    def __init__(self,text,*key):
        self.text = text
        self.key = key
        self.key_length = []
        self.dictionary = {}
        self.alphabet = [chr(i) for i in range(ord('a'),ord('z')+1)]
        self.alphabetC = [chr(i) for i in range(ord('A'),ord('Z')+1)]
        self.dstr = ""
    
    ## load dictionary
    def load_dict(self,filepath):
        with open(filepath) as file:
            for line in file:
                self.dictionary[line.strip().lower()] = 1


    ## crack cypher
    def crackCrypt(self,key_length, method = 's',test_number = 3):
        
        text = self.text
        self.key_length = key_length
        char_count = [dict() for i in range(key_length)]
        char_count_sorted = []
        shift = list(range(key_length))
        
        # collect frequency of each key slice
        for i in range(len(text)):
            c = text[i].lower()
            if c.isalpha():
                try:
                    char_count[i%key_length][c] += 1
                except KeyError:
                    char_count[i%key_length][c] = 1
        
        # Order them
        for c_count in char_count:
            char_count_s = list(c_count.items()) 
            char_count_s.sort(key = lambda item:item[1], reverse = True)
            char_count_sorted.append(char_count_s)
        
        
        if(method == 's'):
            valid_words = {}
            for i in range(pow(test_number,key_length)):
                for k in range(key_length):
                    hf_c = char_count_sorted[k][(i//pow(test_number,key_length-k-1))%test_number][0]
                    shift[k] = ord(hf_c)-ord('e')
                self.key = shift
                self.decrypt()
                valid_wordn = self.test_decrypt() # valid number of words
                valid_words[tuple(shift)] = valid_wordn 
                shift = list(range(key_length))
            
            max_valid_words_set = [(k,v) for (k,v) in valid_words.items() if v == max(valid_words.values())]
            max_valid_words = max_valid_words_set[0][1]            
            self.key = max_valid_words_set[0][0]
            self.decrypt()
            print(f"best key set: {self.key},\n valid words: {max_valid_words}\n")
            
    
        # if(method == 'c'):
           
        #     for i in range(pow(26,key_length)):
        #         for j in range(key_length):
        #             shift.append((i//pow(26,key_length-j-1))%26)
        #         self.key = shift
        #         self.decrypt()
        #         valid_wordn = self.test_decrypt()
        #         valid_words[tuple(shift)] = valid_wordn
        #         shift = []
            
        #     max_valid_words_set = [(k,v) for (k,v) in valid_words.items() if v == max(valid_words.values())]
        #     print(max_valid_words_set)
        #     self.key = max_valid_words_set[0][0]
        #     self.decrypt()          
                
    def find_key_length(self):
        """
        使用卡西斯基检验找到密钥长度。

        Args:
            ciphertext: 密文

        Returns:
            密钥长度
        """
        # 查找密文中重复出现长度至少为 3 的子字符串
        matches = {}
        for i in range(len(self.text) - 3):
            substring = self.text[i:i+3]
            if substring in matches:
                matches[substring].append(i)
            else:
                matches[substring] = [i]
        
        distances = []
        for substring, positions in matches.items():
            if len(positions) > 1:  # 确保至少有两个匹配
                for i in range(len(positions) - 1):
                    distances.append(positions[i + 1] - positions[i])


        # 找出距离的公因子，并选择出现次数最多的公因子作为密钥长度
        factors = Counter()
        for distance in distances:
            for i in range(2, distance + 1):
                if distance % i == 0:
                    factors[i] += 1
                    
        return factors.most_common(5)
    
    def test_decrypt(self):
        valid_words = 0
        str = self.dstr.lower()
        for word in re.split(r"\W+",str):
            #word = word.replace(',','').replace('.','').replace(':','').replace('?','').replace('!','').replace(';','').replace('--','').split('--')
            try:
                valid_words += self.dictionary[word]
            except KeyError:
                continue
        return valid_words
        
        
    def decrypt(self):
        dstr = ""
        for i in range(len(self.text)):
            c = self.text[i]
            if c.isalpha():
                if c in self.alphabet:
                    dstr += self.alphabet[(self.alphabet.index(c)-self.key[i%self.key_length])%26]
                if c in self.alphabetC:
                    dstr += self.alphabetC[(self.alphabetC.index(c)-self.key[i%self.key_length])%26]
            else:
                dstr += c
        self.dstr = dstr  
  

In [11]:
filename = "/home/heisnproph/Project/Test/Duke-Java-Course/Array-list-data/data/secretmessage4.txt"
dictionary_path = "/home/heisnproph/Project/Test/Duke-Java-Course/Array-list-data/data/dictionaries/Danish"
with open(filename,'r') as file:
    str = file.read()
cypher = Cypher(str)
cypher.load_dict(dictionary_path)

In [12]:
print(cypher.find_key_length())

[(2, 43), (4, 39), (3, 29), (6, 25), (12, 21)]


In [18]:
for i in range(12):
    cypher.crackCrypt(i+1,test_number=3)
    dstr = cypher.dstr[0:45]
    print(dstr)

best key set: (4,),
 valid words: 6

Lcrh Srwmy szpgdv ntn.

MCFSM UDFP

Elam eedn
best key set: (15, 4),
 valid words: 8

Acgh Sgwby sopvdk ctc.

BCUSB JDUP

Tlpm etdc
best key set: (12, 7, 4),
 valid words: 8

Dzrz Sjtmq srmgvs fqn.

EZFKJ MAFH

Wiae ewan
best key set: (3, 12, 15, 17),
 valid words: 15

Mugu Kgjnq fahvqw cgo.

NUUFN JQGH

Fdpz wtqo
best key set: (3, 0, 12, 15, 12),
 valid words: 10

Mgjw Tvobq wreyez clo.

EDJKB VHXE

Idpe iwsf
best key set: (-1, 17, -2, 12, 18, 12),
 valid words: 12

Qpxz Kwjsq kecmvh sgt.

RPLKY ZQLH

Jyge wjqt
best key set: (21, 2, 19, 0, 0, 15, 4),
 valid words: 12

Uecl Hrfoj woppfg rin.

QGUSV FHJE

Gweq enfy
best key set: (18, 12, 15, 17, 18, 15, 15, 17),
 valid words: 14

Xugu Hgjyq flevqh cgz.

YUUFY JQRH

Qapz wtqz
best key set: (11, 17, 17, -1, 12, 15, 17, 6, 15),
 valid words: 11

Epem Heubr fehvqt gga.

ZAULZ ZVUC

Xynr trbc
best key set: (18, 12, 18, -1, 18, 16, 15, 17, 13, 12),
 valid words: 13

Xudm Ggjdq klusrk elz.

YQUFD GVRU

Tyre

KeyboardInterrupt: 

In [ ]:
dictionary = {}

with open("/home/heisnproph/Project/Test/Duke-Java-Course/Array-list-data/data/dictionaries/English") as f:
    for line in f:
        dictionary[line.strip()] = 1

for i in dictionary:
    print(i)

In [174]:
from time import time

def decypher(key_length):
    start_time = time()
    filename = "/home/heisnproph/Project/Test/Duke-Java-Course/Array-list-data/data/secretmessage2.txt"
    dictionary_path = "/home/heisnproph/Project/Test/Duke-Java-Course/Array-list-data/data/dictionaries/"
    with open(filename,'r') as file:
        str = file.read()
    cypher = Cypher(str)
    cypher.load_dict(dictionary_path)
    cypher.crackCrypt(key_length,test_number=1)
    dstr = cypher.dstr[0:40]
    print(dstr)
    elapsed_time = time() - start_time
    print(f"coomputing time: {elapsed_time:.2f} \n key length: {key_length} \n\n")

    


In [231]:
filename = "/home/heisnproph/Project/Test/Duke-Java-Course/Array-list-data/data/secretmessage2.txt"
dictionary_path = "/home/heisnproph/Project/Test/Duke-Java-Course/Array-list-data/data/dictionaries/English"
with open(filename,'r') as file:
    str = file.read()
cypher = Cypher(str)
cypher.load_dict(dictionary_path)




best key set: (8, -3, 20, 14, -3, 10, 9, 6, -1, 4, 18, 16, 12, 15, 11, 19, 13, 21, 4, 8, 15, 10, 14, -3, 3, 9, 6, -1, 4, 18, 16, 12, 19, 11, 19, 0, 21, 4),
 valid words: 4324

Exu Jvwgety of Hnewat, Loyysr or Duaxqrv

AIQ


In [222]:
print(cypher.test_decrypt())

3524


In [175]:
from multiprocessing import Pool

with Pool(processes=13) as pool:
    for i in range(100):
        pool.apply_async(func=decypher,args=(i+1,))
       
    pool.close()
    pool.join()



best key set: (9, 19, 4, 19, 19, 4, -3),
 valid words: 2847
Dbk Nbidktx wx Hwghel, Eyoaum ak Kqrzqgz
coomputing time: 0.33 
 key length: 7 


best key set: (19, 19, 4, 19, 9, 19, 4, 19),
 valid words: 2908best key set: (19, 19),
 valid words: 2814

Tbk Xmbtkem rb Hhgwpz, Pyoaym ak VjhjbvzTbv Nmmtkex hb Hhghpp, Pyolyc lk Vuhzbgz

coomputing time: 0.35 
 key length: 2 

coomputing time: 0.35 
 key length: 8 



best key set: (19, 4, -3, 19, 9, -3, 19, 21, -3, 19, 4, -3),
 valid words: 2885
Tqr Ximrgem hq Hrchnl, Lydhym li Vjdzqcz
coomputing time: 0.36 
 key length: 12 


best key set: (4, 19, 19, 15, 19, 19, 19, 19, 4, 4),
 valid words: 2867best key set: (4, 19, 19, 15, 4),
 valid words: 2802

Ibv Nmmtztm hf Hhgwee, Tyolyr ak ZuhzbvoIbv Cbmtotm hf Whglee, Tndlyg ak Zjwzbko

coomputing time: 0.37 
 key length: 10 

coomputing time: 0.37 
 key length: 5 



best key set: (19,),
 valid words: 2814
Tbv Nmmtkex hb Hhghpp, Pyolyc lk Vuhzbgz
coomputing time: 0.38 
 key length: 1 


best key set